保存 OpenAI API Key 為環境變量

In [1]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
# 兩個 API 的密鑰
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [2]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

# 建立系統消息和用戶消息的 ChatMessage 對象
messages = [
    ChatMessage.from_system(
        "即使某些輸入資料採用其他語言，也始終以繁體中文回應。"
    ),
    ChatMessage.from_user(
        "什麼是自然語言處理？要簡潔。"
    ),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(model="gpt-4-turbo")
# 傳入消息並運行
chat_generator.run(messages=messages)

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'replies': [ChatMessage(content='自然語言處理（Natural Language Processing，縮寫為NLP）是一門跨領域的科學技術，主要研究如何讓計算機自動地理解和處理人類語言的方法和技術。這包括從語言學基礔到複雜的演算法和模型，目的在於實現與人類語言相關的各程自動化任務，例如語音識別、機器翻譯、情感分析以及自動摘要等。', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4-turbo-2024-04-09', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 174, 'prompt_tokens': 71, 'total_tokens': 245}})]}

In [3]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 使用流式回調函數初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息並運行
response = chat_generator.run(messages=messages)

自然語言虠理（Natural Language Processing, NLP）是計算機科學、人工智能與語言學領域的交叉學科，旨在讓計算機能夠理解、解釋和生成人類語言。透過NLP，機器可以執行語音識別、機器翻譯、情感分析等任務。

In [4]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

# 建立文件
# documents = [
#     Document(content="我的名字是 Jean，我住在 Paris。"),
#     Document(content="我的名字是 Mark，我住在 Berlin。"),
#     Document(content="我的名字是 Giorgio，我住在 Rome。"),
#     Document(content="我的名字是 Marta，我住在 Madrid。"),
#     Document(content="我的名字是 Harry，我住在 London。"),
# ]
documents = [
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome."),
    Document(content="My name is Marta and I live in Madrid."),
    Document(content="My name is Harry and I live in London."),
]
# 初始化內存文件儲存
document_store = InMemoryDocumentStore()

# 建立索引管道
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"), name="doc_embedder"
)
indexing_pipeline.add_component(
    instance=DocumentWriter(document_store=document_store),
    name="doc_writer"
)

# 連接嵌入器和文件寫入器
indexing_pipeline.connect(
    "doc_embedder.documents",
    "doc_writer.documents"
)

# 運行管道
indexing_pipeline.run({
    "doc_embedder": {"documents": documents}
})

/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:174: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v3 of SentenceTransformers.
  warnings.warn(
/Users/samhsiao/Documents/PythonVenv/envHaystack/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.73it/s]


{'doc_writer': {'documents_written': 5}}

In [5]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

# 定義提示模板
template = """
根據給定的上下文回答問題。

上下文:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
問題: {{ question }}
答案:
"""

# 建立 RAG 管道
rag_pipe = Pipeline()
rag_pipe.add_component(
    "embedder",
    SentenceTransformersTextEmbedder(
        model="sentence-transformers/all-MiniLM-L6-v2"
    )
)
rag_pipe.add_component(
    "retriever",
    InMemoryEmbeddingRetriever(
        document_store=document_store
    )
)
rag_pipe.add_component(
    "prompt_builder",
    PromptBuilder(
        template=template
    )
)
rag_pipe.add_component(
    "llm",
    OpenAIGenerator(model="gpt-4-turbo")
)

# 連接組件
rag_pipe.connect(
    "embedder.embedding",
    "retriever.query_embedding"
)
rag_pipe.connect(
    "retriever",
    "prompt_builder.documents"
)
rag_pipe.connect(
    "prompt_builder",
    "llm"
)

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
query = "Mark 住在哪裡？"
rag_pipe.run({
    "embedder": {"text": query},
    "prompt_builder": {"question": query}
})

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.82it/s]


{'llm': {'replies': ['Mark 住在 Berlin.'],
  'meta': [{'model': 'gpt-4-turbo-2024-04-09',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 7,
     'prompt_tokens': 107,
     'total_tokens': 114}}]}}

In [7]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({
        "embedder": {"text": query},
        "prompt_builder": {"question": query}
    })
    return {"reply": result["llm"]["replies"][0]}

In [8]:
WEATHER_INFO = {
    "Berlin": {
        "weather": "mostly sunny", "temperature": 7, "unit": "celsius"
    },
    "Paris": {
        "weather": "mostly cloudy", "temperature": 8, "unit": "celsius"
    },
    "Rome": {
        "weather": "sunny", "temperature": 14, "unit": "celsius"
    },
    "Madrid": {
        "weather": "sunny", "temperature": 10, "unit": "celsius"
    },
    "London": {
        "weather": "cloudy", "temperature": 9, "unit": "celsius"
    },
}

def get_current_weather(location: str):
    if location in WEATHER_INFO:
        return WEATHER_INFO[location]
    else:
        # 回退數據
        return {
            "weather": "狂風暴雨",
            "temperature": 99.9,
            "unit": "fahrenheit"
        }


In [9]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "rag_pipeline_func",
            "description": "獲取有關人們居住地點的信息",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "搜尋中使用的查詢。從用戶的消息中推斷出這一點。它應該是一個問題或一個陳述。",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "取得當前天氣",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市和州，例如加州舊金山"
                    }
                },
                "required": ["location"],
            },
        },
    },
]

In [10]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 建立消息列表，包含系統消息和用戶查詢
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。如果用戶要求不明確，請要求澄清。"
    ),
    ChatMessage.from_user("你能告訴我 Mark 住在哪裡嗎？"),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息和工具列表並運行
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)
# 輸出查看
print(response)

{'replies': [ChatMessage(content='[{"index": 0, "id": "call_PK1299vodTJ4uFSyAUVXp741", "function": {"arguments": "{\\"query\\":\\"Where does Mark live?\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-4-turbo-2024-04-09', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {}})]}


In [11]:
import json

# 解析函數調用信息
# 提取第一個回應中的 content
content = response['replies'][0].content

# 將 content 解析為 JSON
# content 是一個 JSON 字串，需要轉換為 Python 字典
function_calls = json.loads(content)

# 提取第一個函數調用信息
# 提取函數調用列表中的第一個元素
function_call = function_calls[0]

# 獲取函數名稱
# 獲取函數名稱，這是我們需要調用的函數
function_name = function_call['function']['name']

# 解析函數參數
# 將參數解析為字典格式
function_args = json.loads(function_call['function']['arguments'])

# 打印函數名稱和參數
print("Function Name:", function_name)
print("Function Arguments:", function_args)


# 可用函數字典
available_functions = {
    "rag_pipeline_func": rag_pipeline_func,
    "get_current_weather": get_current_weather
}

# 搜尋相應的函數並使用給定的參數調用它
if function_name in available_functions:
    # 根據函數名稱找到對應的函數
    function_to_call = available_functions[function_name]
    # 使用解包操作將參數傳遞給函數
    function_response = function_to_call(**function_args)
    # 打印函數的返回值
    print("Function Response:", function_response)
else:
    # 如果函數名稱未找到，打印錯誤訊息
    print(f"Function {function_name} not found.")

Function Name: rag_pipeline_func
Function Arguments: {'query': 'Where does Mark live?'}


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.72it/s]


Function Response: {'reply': 'Berlin'}


In [12]:
from haystack.dataclasses import ChatMessage

# 建立函數回應消息
function_message = ChatMessage.from_function(
    content=json.dumps(function_response),
    name=function_name
)
# 將函數回應消息添加到消息列表
messages.append(function_message)

# 再次運行 OpenAIChatGenerator
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)

Mark 住在柏林。

In [13]:
import gradio as gr
import json

from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

chat_generator = OpenAIChatGenerator(model="gpt-4-turbo")
response = None
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。"
        "如果用戶要求不明確，請要求澄清。"
    )
]

# 定義聊天機器人函數
def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

    while True:
        # 如果 OpenAI 回應是一個工具調用
        if response and response["replies"][0].meta["finish_reason"] == "tool_calls":
            function_calls = json.loads(response["replies"][0].content)
            print(response["replies"][0])
            for function_call in function_calls:
                # 解析函數調用信息
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                # 搜尋相應的函數並使用給定的參數調用它
                function_to_call = available_functions[function_name]
                function_response = function_to_call(function_args)

                # 使用 `ChatMessage.from_function` 將函數回應添加到消息列表
                messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

        # 常規對話
        else:
            messages.append(response["replies"][0])
            break
    return response["replies"][0].content

# 建立聊天界面
demo = gr.ChatInterface(
    fn=chatbot_with_fc,
    # 顯示在下方的範例欄位
    examples=[
        "你能告訴我 Giorgio 住在哪裡嗎？",
        "Madrid 的天氣怎麼樣？",
        "誰住在 London?",
        "Mark 住的地方的天氣怎麼樣？",
    ],
    title="請詢問有關天氣或人們居住的地方。",
)

In [14]:
# 啟動聊天應用程序
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
